In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [4]:
import os 

corpus = []

def read_text_file(file_path): 
    with open(file_path, 'r') as f: 
        file = f.read()
        for line in file.split("\n"):
            corpus.append(line)
for file in os.listdir():
    if file.endswith(".txt"): 
        file_path = f"{file}"
        read_text_file(file_path) 
print(len(corpus))

3186


In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print(total_words)

5763


In [7]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [8]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = "pre"))
print(input_sequences[50:100])

[[   0    0    0    0    0    0    0    0    2   40    9    1  609    3
     4]
 [   0    0    0    0    0    0    0    2   40    9    1  609    3    4
  1921]
 [   0    0    0    0    0    0    2   40    9    1  609    3    4 1921
   405]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0  610
     4]
 [   0    0    0    0    0    0    0    0    0    0    0    0  610    4
   264]
 [   0    0    0    0    0    0    0    0    0    0    0  610    4  264
     2]
 [   0    0    0    0    0    0    0    0    0    0  610    4  264    2
   331]
 [   0    0    0    0    0    0    0    0    0  610    4  264    2  331
     4]
 [   0    0    0    0    0    0    0    0  610    4  264    2  331    4
   332]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    2
  1134]
 [   0    0    0    0    0    0    0    0    0    0    0    0    2 1134
    45]
 [   0    0    0    0    0    0    0    0    0    0    0    2 1134   45
    21]
 [   0    0    0    0    0    0    0    

In [9]:
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes = total_words)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 240, input_length = max_sequence_len - 1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(total_words, activation = "softmax")
])

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

2024-07-03 00:00:29.605205: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-03 00:00:29.605965: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-03 00:00:29.606474: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [14]:
history = model.fit(xs, ys, epochs = 30, verbose = 1)

Epoch 1/30
174/555 [========>.....................] - ETA: 7s - loss: 0.5105 - accuracy: 0.8592